In [16]:
import requests
from bs4 import BeautifulSoup
from pathlib import Path
import re
import json
from urllib.parse import urljoin
import pandas as pd
import time
import math
from rich import print
from rich.progress import track
from random import randrange

In [17]:
## ID number for url
#97376 

In [18]:
def get_url(
    url: str,
    user_agent: str = "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36",
    timeout: int = 30,
    verbose: bool = False,
) -> requests.Response:
    """Get the provided URL.

    Args:
        url (str): The URL to get.
        user_agent (str, optional): The user agent to use. Defaults to "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/
        timeout (int, optional): The number of seconds to wait before timing out. Defaults to 30.
        verbose (bool, optional): Whether to print out the results. Defaults to False.

    Returns:
        requests.Response: The response object.
    """
    # Set the headers
    headers = {
        "User-Agent": user_agent,
    }

    # Log the URL, if verbose
    if verbose:
        print(f"Fetching {url}")

    # Get the URL
    r = requests.get(url, headers=headers, timeout=timeout)

    # Raise an error if the request failed
    assert r.ok

    # Return the response
    return r

In [19]:
#the url for the site
url= "https://store.steampowered.com/search/?sort_by=Released_DESC&force_infinite=1&tags=97376&category1=998&supportedlang=english&ndl=1"

In [20]:
r = get_url(url, verbose=True)

Fetching 
https://store.steampowered.com/search/?sort_by=Released_DESC&force_infinite=1&tags=97376&category1=998&supportedlan
g=english&ndl=1

In [21]:
soup = BeautifulSoup(r.text, "html.parser")

In [22]:
def timer(start_time,end_time):
    """
    This is a function for a timer
    para1 = int for start time
    para2 = int for end time
    """
    snoozer = randrange(start_time,end_time)
    time.sleep(snoozer)

In [23]:
#total number of games in the category 10/25/24
total_games = 572

In [24]:
# houw many games per page
per_page = 50

In [25]:
#get the amount of pages to scrape rounded
page_count = math.ceil(total_games / per_page)
page_count

12

In [26]:
#enter that into its own variable
page_list = list(range(0, page_count))

In [27]:
#set path for raw data to go to
json_dir = Path() / "output" / "json" / "cozy"
json_dir

PosixPath('output/json/cozy')

In [28]:
#scrape raw data from page and save it to the folder
for i in track(page_list):
    # If the JSON for this page is already downloaded, skip it!
    output_path = json_dir / f"{i}.json"
    if output_path.exists():
        continue

    # Otherwise, download it!
    start = i * 50
    url = f"https://store.steampowered.com/search/results/?query&start={start}&count=50&dynamic_data=&sort_by=Released_DESC&snr=1_7_7_240_7&tags=97376&category1=998&supportedlang=english&infinite=1"
    r = requests.get(url)
    assert r.ok
    
    # Now save it out!
    data = r.json()
    with open(output_path, "w") as f:
        json.dump(data, f, indent=2)

    # Throw in a delay
    timer(5,12)

Output()

In [29]:
json_files = list(json_dir.glob("*.json"))

In [30]:
# parse through raw data and get information needed
parsed_list = []
for j in track(json_files):
    data = json.load(open(j, "r"))
    soup = BeautifulSoup(data["results_html"])
    for ele in soup.find_all(attrs={'class':"responsive_search_name_combined"}):
        d = {}

        d['title'] = ele.find(attrs={"class": "title"}).text.strip()
        d['release_date'] = ele.find("div", {"class": "search_released"}).text.strip()
        d['final_price'] = ele.find("div", {"class": "search_price_discount_combined"})['data-price-final']

        try:
            d['review_summary'] = ele.find("span", {"class": "search_review_summary"})["data-tooltip-html"]
        except TypeError:
            d['review_summary'] = None
        
        try:
            d['original_price'] = ele.find("div", {"class": "discount_original_price"}).text
        except (AttributeError, TypeError):
            d['original_price'] = None
        
        try:   
            d['discount_price'] = ele.find("div", {"class": "discount_final_price"}).text.strip()
        except AttributeError:
            d['discount_price'] = None
        
        parsed_list.append(d)

Output()

In [32]:
#save to dataframe
df = pd.DataFrame(parsed_list)
df

,title,release_date,final_price,review_summary,original_price,discount_price
0,CottageVille,"Nov 1, 2023",699,None,None,$6.99
1,Viking Chef: Feast Frenzy,"Oct 31, 2023",499,None,None,$4.99
2,Doodle Harmony,"Oct 30, 2023",199,Positive<br>96% of the 28 user reviews for thi...,None,$1.99
3,Twofold,"Oct 27, 2023",1999,Very Positive<br>95% of the 63 user reviews fo...,None,$19.99
4,Terra Toy,"Oct 27, 2023",499,Positive<br>93% of the 15 user reviews for thi...,None,$4.99
...,...,...,...,...,...,...
567,Railway Islands - Puzzle,"Nov 18, 2021",71,Very Positive<br>96% of the 198 user reviews f...,$3.99,$0.71
568,Dogs Organized Neatly,"Nov 1, 2021",149,"Overwhelmingly Positive<br>98% of the 1,076 us...",$2.99,$1.49
569,Luffy: Way Back Home,"Oct 21, 2021",199,Positive<br>85% of the 20 user reviews for thi...,None,$1.99
570,Dinogotchi,"Sep 30, 2021",249,Very Positive<br>92% of the 55 user reviews fo...,None,$2.49


In [33]:
#make sure it's all there
assert len(df) == total_games

In [34]:
df.sample(15)

,title,release_date,final_price,review_summary,original_price,discount_price
418,The Cat Maze,"Sep 27, 2022",499,None,None,$4.99
506,Railway Islands 2 - Puzzle,"Nov 20, 2023",499,Positive<br>95% of the 20 user reviews for thi...,None,$4.99
500,William and Sly: Classic Collection,"Dec 1, 2023",0,Very Positive<br>97% of the 145 user reviews f...,None,Free
111,Coloring Game 4,"Jul 30, 2021",0,"Very Positive<br>93% of the 1,779 user reviews...",None,Free
450,Cats and Seek: Dino Park,"Feb 25, 2024",199,Very Positive<br>100% of the 139 user reviews ...,None,$1.99
372,A Weekend in Puzzleburg,"Feb 17, 2023",1999,None,None,$19.99
329,Duck Detective: The Secret Salami,"May 23, 2024",999,"Overwhelmingly Positive<br>95% of the 3,314 us...",None,$9.99
206,Frog World,"Aug 15, 2024",0,Positive<br>96% of the 26 user reviews for thi...,None,Free
268,Snake Pass,"Mar 28, 2017",1999,"Very Positive<br>86% of the 1,054 user reviews...",None,$19.99
333,A Tower Full of Cats,"May 20, 2024",479,"Overwhelmingly Positive<br>99% of the 1,262 us...",$5.99,$4.79


In [35]:
#set a path for spreadsheet to go 
excel_path= Path() / "sheets"
excel_path

PosixPath('sheets')

In [36]:
#name the file
file_name= "cozyGamesData.xlsx"

In [37]:
# combine above information
file_path= excel_path/file_name

In [38]:
#save df to an excel spreadsheet
df.to_excel(file_path,index=False)